In [63]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
import factor_analyzer
from factor_analyzer import FactorAnalyzer
pd.options.display.float_format = '{:.2f}'.format

In [64]:
#loading all the data
dfc=pd.read_csv('dfc.csv')
dfv=pd.read_csv('dfv.csv')
dfv_ColNames=pd.read_csv('dfv_ColNames.csv')
dfv_noDupe_ColNames=pd.read_csv('dfv_noDupe_ColNames.csv')
dfvnoDupe_recode=pd.read_csv('dfv_noDupe_recode.csv')
dfv_noDupe=pd.read_csv('dfv_noDupe.csv')
dfv_recode=pd.read_csv('dfv_recode.csv')
dfv_original=pd.read_csv('dfV_original.csv')
dfc_original=pd.read_csv('dfC_orignial.csv')

In [65]:
dfc.Case= dfc.Case.astype('category')
dfv.Case= dfv.Case.astype('category')

In [66]:
dfc.columns#errors in Publisher values

Index(['Case', 'CommCase', 'Platform', 'Publisher', 'VidTitle', 'CommentText',
       'CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus',
       'Alignment', 'MediaType'],
      dtype='object')

In [67]:
dfv.columns

Index(['Case', 'Platform', 'Publisher', 'VideoURL', 'PubDate', 'Likes',
       'Comments', 'Saves', 'Shots', 'Graphics', 'EditsStyle', 'Broll',
       'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive',
       'Sourcing', 'Fairness', 'Agenda', 'MediaIntegrity', 'Decorum',
       'Language', 'Topic', 'Presentation', 'ExternalVidSource', 'MediaType',
       'Animation', 'Filters', 'Decorum_Z', 'Language_Z'],
      dtype='object')

In [68]:
# merge dfc and selected columns from dfv_noDupe
dfc = pd.merge(dfc, dfv, on=['Case'], how='inner', indicator=True)

In [69]:
trendiness = dfv[['Humor', 'Animation', 'Filters', 'Music']]
pg.cronbach_alpha(data=trendiness)

(0.6575492422021552, array([0.548, 0.746]))

In [70]:
trendinessAll = dfv_noDupe[['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendiness1)

(0.2463850159344485, array([0.01 , 0.447]))

In [71]:
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.3477652944172873, array([0.128, 0.528]))

In [72]:
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.3632573521019634, array([0.142, 0.541]))

In [73]:
trendiness = dfv_noDupe[['EditsStyle','Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.35102439590202794, array([0.132, 0.53 ]))

In [74]:
#EFA
fa=FactorAnalyzer(n_factors=6, rotation="varimax")#setting terms
fa.fit(trendinessAll)#applying terms for df with target variables
ev, v = fa.get_eigenvalues()#eigenvalues
EFAloaging=pd.DataFrame(fa.loadings_)#creating EFA loading df
EFAloaging.to_csv('EFAloaging.csv')#saving EFA loading df
EFAloaging

,0,1,2,3,4,5
0,0.16,-0.17,0.07,-0.06,0.00,0.60
1,0.06,0.16,0.76,-0.09,0.07,0.14
2,0.16,-0.35,0.56,-0.16,0.45,-0.58
3,0.42,-0.22,0.07,-0.25,0.04,-0.28
4,-0.03,0.64,0.33,-0.02,-0.00,-0.23
5,0.03,0.17,0.56,0.14,0.22,-0.09
6,-0.06,0.31,0.36,-0.13,0.79,-0.04
7,-0.26,-0.00,0.10,0.94,0.11,0.16
8,-0.01,-0.08,0.03,-0.32,0.11,0.09
9,-0.13,0.62,-0.02,0.09,0.01,0.02


In [75]:
pd.DataFrame(ev)#factor variance

,0
0,2.79
1,2.72
2,1.59
3,1.27
4,1.14
5,0.70
6,0.68
7,0.63
8,0.49
9,0.42


In [76]:
trendiness2 = trendiness1.copy()
trendiness2.columns = [str(i) + '_' + col for i, col in enumerate(trendiness1.columns)]
trendiness2.columns

Index(['0_Shots', '1_EditsStyle', '2_Broll', '3_NatSound', '4_Audio',
       '5_Music', '6_Narration', '7_Humor', '8_Sourcing', '9_Decorum',
       '10_Language', '11_Animation', '12_Filters'],
      dtype='object')

In [78]:
#observed factors
factor0=['NatSound', 'Animation', 'Filters']
factor1=['Audio', 'Decorum', 'Language']
factor2=['EditsStyle', 'Broll', 'Music']
factor4=['Broll', 'Music', 'Narration']
#creating a factorlist
factorlist = [factor0, factor1, factor2, factor4]
for factor in factorlist:
    Cronbach = dfv_noDupe[factor]
    print(factor)
    print(pg.cronbach_alpha(data=Cronbach))

['NatSound', 'Animation', 'Filters']
(0.8157673667205168, array([0.743, 0.871]))
['Audio', 'Decorum', 'Language']
(0.4376833889998119, array([0.216, 0.605]))
['EditsStyle', 'Broll', 'Music']
(0.6577181208053693, array([0.523, 0.759]))
['Broll', 'Music', 'Narration']
(0.690359057199501, array([0.568, 0.782]))


In [ ]:
dfv.Platform.value_counts()

Platform
YouTube Shorts     42
Instagram Reels    42
TikTok             42
Name: count, dtype: int64

RQ1: Does short-form video news lean more trendy or more traditional?

H1: Trendiness is more prevalent on TikTok compared to YouTube Shorts and Instagram Reels.

In [ ]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['trendiness']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,97,1.51,0.23,0.03


In [ ]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,Platform,Instagram Reels,TikTok,False,True,-0.65,63.00,two-sided,0.52,1.00,bonf,0.304,-0.16
1,trendiness,Platform,Instagram Reels,YouTube Shorts,False,True,1.08,68.00,two-sided,0.28,0.85,bonf,0.405,0.26
2,trendiness,Platform,TikTok,YouTube Shorts,False,True,1.74,62.76,two-sided,0.09,0.26,bonf,0.906,0.42


In [ ]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Instagram Reels,-30.77,67.05
1,trendiness,TikTok,-20.80,57.55
2,trendiness,YouTube Shorts,-48.69,71.62



RQ2: Are people more positive towards a topic when news is presented more trendy or more traditional?

H2: People are more positive towards a topic when the presentation is trendy.


RQ3: When news is presented in traditional formats, do people engage less?

H3: Traditional presentations lead to less engagement online.


RQ4: Does post engagement occur more from positive, neutral, or negative comments?

H4: Negative comments reflect more engagements and interactions on posts.